In [1]:
import hashlib
def sha1(fname):
    sha1hash = hashlib.sha1()
    with open(fname) as handle: #opening the file one line at a time for memory considerations
        for line in handle:
            sha1hash.update(line.encode('utf-8'))
    return(sha1hash.hexdigest())
    
def comparefiles(files,datakey):
    print('########################################')
    print('Checking the files\' hash.')
    if sha1(files[0]) == sha1(files[1]):
        print('100% Match')
    else:
        print('Hash not matched. Proceeding to detailed checks.')
comparefiles(files, datakey)

NameError: name 'files' is not defined

In [2]:
comparefiles(['dask1', 'dask2'], '')

########################################
Checking the files' hash.


FileNotFoundError: [Errno 2] No such file or directory: 'dask1'